# Evaluate Coefficients of Logistic Regression

[Machine Learning Interpretability course](https://www.trainindata.com/p/machine-learning-interpretability)

In this notebook, we will evaluate the coefficients of the logistic regression. We will first use `statsmodels` and then `sklearn`.

`statsmodels` returns the statistics and p-values out of the box, utilizing MLE. For `sklearn`, we will obtain these parameters manually, also using the derivations from MLE.

If the coefficients are significant, then the contribution of the feature towards the probability is meaningful.

In [1]:
import numpy as np
import pandas as pd
from scipy import stats

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm

### Load data

To obtain the data, check the folder `prepare-data` in this repo or section 2 in the course.

In [2]:
# load titanic dataset

df = pd.read_csv('../titanic.csv')

# split data
X_train, X_test, y_train, y_test = train_test_split(
    df.drop("survived", axis=1), 
    df["survived"],
    test_size=0.15,
    random_state=1,
)

# scale the variables
scaler = StandardScaler().set_output(transform="pandas")

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Statsmodels

In [3]:
# Our model needs an intercept so we add a column of 1s:

X_train_sm = sm.add_constant(X_train)
X_test_sm = sm.add_constant(X_test)

In [4]:
# Fit logistic regression model

logit_mod = sm.Logit(y_train, X_train_sm)

logit_res = logit_mod.fit()

print(logit_res.summary())

Optimization terminated successfully.
         Current function value: 0.469479
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:               survived   No. Observations:                 1112
Model:                          Logit   Df Residuals:                     1098
Method:                           MLE   Df Model:                           13
Date:                Tue, 14 Nov 2023   Pseudo R-squ.:                  0.2923
Time:                        10:56:39   Log-Likelihood:                -522.06
converged:                       True   LL-Null:                       -737.67
Covariance Type:            nonrobust   LLR p-value:                 5.606e-84
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.6505      0.079     -8.267      0.000      -0.805      -0.496
pclass        -0.5174      0.

With statsmodels, the statistical testing of the coefficients comes out of the box.

We see that some coefficients are not statistically significant, so we could remove those features from the model.

## Scikit-learn

In [5]:
# fit model

logit = LogisticRegression(penalty=None, random_state=1)

logit.fit(X_train, y_train)

LogisticRegression(penalty=None, random_state=1)

### Empirically

We follow the derivations shown [here](https://web.stanford.edu/class/archive/stats/stats200/stats200.1172/Lecture26.pdf) and the Python implementation shown [here](https://stats.stackexchange.com/questions/89484/how-to-compute-the-standard-errors-of-a-logistic-regressions-coefficients).

In [6]:
# the target classes

logit.classes_

array([0, 1], dtype=int64)

In [7]:
# matrix of predicted class probabilities

pred_train = logit.predict_proba(X_train)

pred_train

array([[0.89651014, 0.10348986],
       [0.76111387, 0.23888613],
       [0.55621831, 0.44378169],
       ...,
       [0.93911299, 0.06088701],
       [0.71414115, 0.28585885],
       [0.41312738, 0.58687262]])

In [8]:
# add column of 1's at the beginning of X_train

X_train_c = np.hstack([np.ones((X_train.shape[0], 1)), X_train])

X_train_c

array([[ 1.        ,  0.83427143, -0.47413915, ..., -0.12827074,
         0.5399273 , -0.06720649],
       [ 1.        , -0.36742524,  0.46905   , ..., -0.12827074,
         0.5399273 , -0.06720649],
       [ 1.        ,  0.83427143,  2.35542828, ..., -0.12827074,
         0.5399273 , -0.06720649],
       ...,
       [ 1.        ,  0.83427143,  2.35542828, ..., -0.12827074,
         0.5399273 , -0.06720649],
       [ 1.        , -1.56912191, -0.47413915, ..., -0.12827074,
         0.5399273 , -0.06720649],
       [ 1.        ,  0.83427143, -0.47413915, ..., -0.12827074,
         0.5399273 , -0.06720649]])

In [9]:
# Create matrix of 0's, where diagonal contains each 
# predicted observation's variance.

W = np.diagflat(np.product(pred_train, axis=1))

W

array([[0.09277971, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.18181955, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.2468395 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.05717978, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.20414357,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.24245315]])

In [10]:
# The variance is the product of the probabilities

# For the first observation:
0.89651014 * 0.10348986

0.0927797088771804

In [11]:
# Covariance matrix, it is derived empirically.

# (XT W X)−1

cov_m = np.linalg.inv(np.dot(np.dot(X_train_c.T, W), X_train_c))

cov_m

array([[ 6.19148576e-03,  9.50979865e-04,  4.04960853e-04,
        -1.28885132e-04, -9.73906357e-04,  3.31124990e-04,
         1.25713235e-04,  7.24502525e-04,  7.11303716e-04,
         5.67242165e-04,  4.37754332e-04,  1.75871235e-04,
         8.61118068e-04,  3.04792119e-05],
       [ 9.50979865e-04,  1.38932240e-02, -7.85293102e-04,
        -5.80503656e-04, -5.68088469e-04,  3.37760169e-03,
         3.12774306e-03,  5.16264693e-03,  6.33396731e-03,
         3.41794219e-03,  3.88078452e-03,  2.94650855e-03,
        -3.87665730e-04, -2.43197064e-04],
       [ 4.04960853e-04, -7.85293102e-04,  9.05920372e-03,
        -2.29035784e-03, -1.17425375e-03, -1.26791128e-03,
        -7.73044204e-04, -3.44877875e-04, -9.14102587e-04,
        -1.31800356e-04, -2.24792355e-04, -5.39609035e-05,
        -1.18947160e-04,  2.45175335e-04],
       [-1.28885132e-04, -5.80503656e-04, -2.29035784e-03,
         6.72920647e-03, -1.26879216e-03, -1.42280469e-03,
        -1.32916565e-03, -1.39077749e-04,  2.

In [12]:
# Standard errors

se = np.sqrt(np.diag(cov_m))

se

array([0.07868599, 0.11786952, 0.09517985, 0.08203174, 0.08207417,
       0.12548844, 0.12746087, 0.14865534, 0.17037341, 0.12939522,
       0.13046903, 0.09959975, 0.2364004 , 0.07336058])

In [13]:
# capture name of features

features = np.insert(logit.feature_names_in_, 0, "intercept")

features

array(['intercept', 'pclass', 'sibsp', 'parch', 'sex_female',
       'embarked_S', 'embarked_C', 'cabin_B', 'cabin_C', 'cabin_E',
       'cabin_D', 'cabin_A', 'cabin_M', 'cabin_G'], dtype=object)

In [14]:
# capture coefficients of the logit including the intercept

coefficients = np.insert(logit.coef_[0], 0, logit.intercept_[0])

coefficients

array([-0.65051925, -0.51737413, -0.22689809,  0.05420292,  1.19989943,
       -0.00845913,  0.23404885, -0.21982215, -0.37650021, -0.02520391,
       -0.19821006, -0.1050838 , -0.68045168, -0.14362905])

In [15]:
# Wald's statistic (coefficient / s.e.)

z_scores = (coefficients / np.sqrt(np.diag(cov_m)))

z_scores

array([-8.26728127, -4.38938008, -2.38388788,  0.66075542, 14.61969636,
       -0.06740967,  1.83624077, -1.47873705, -2.20985305, -0.19478241,
       -1.5192116 , -1.05506088, -2.8783864 , -1.9578506 ])

In [16]:
# p-values

p_values = stats.norm.sf(abs(z_scores))*2  # two sided

p_values

array([1.37048362e-16, 1.13674245e-05, 1.71308208e-02, 5.08769178e-01,
       2.10336291e-48, 9.46255570e-01, 6.63220556e-02, 1.39210605e-01,
       2.71153623e-02, 8.45563294e-01, 1.28709242e-01, 2.91397480e-01,
       3.99715215e-03, 5.02475461e-02])

In [17]:
# put everything together

coeffs = pd.DataFrame({
    "feature": features,
    "coeff": coefficients,
    "std": se,
    "z": z_scores,
    "p_value": p_values,
})

coeffs

,feature,coeff,std,z,p_value
0,intercept,-0.650519,0.078686,-8.267281,1.370484e-16
1,pclass,-0.517374,0.117870,-4.389380,1.136742e-05
2,sibsp,-0.226898,0.095180,-2.383888,1.713082e-02
3,parch,0.054203,0.082032,0.660755,5.087692e-01
4,sex_female,1.199899,0.082074,14.619696,2.103363e-48
5,embarked_S,-0.008459,0.125488,-0.067410,9.462556e-01
6,embarked_C,0.234049,0.127461,1.836241,6.632206e-02
7,cabin_B,-0.219822,0.148655,-1.478737,1.392106e-01
8,cabin_C,-0.376500,0.170373,-2.209853,2.711536e-02
9,cabin_E,-0.025204,0.129395,-0.194782,8.455633e-01


In [18]:
# Compare with statsmodels

print(logit_res.summary())

                           Logit Regression Results                           
Dep. Variable:               survived   No. Observations:                 1112
Model:                          Logit   Df Residuals:                     1098
Method:                           MLE   Df Model:                           13
Date:                Tue, 14 Nov 2023   Pseudo R-squ.:                  0.2923
Time:                        10:56:40   Log-Likelihood:                -522.06
converged:                       True   LL-Null:                       -737.67
Covariance Type:            nonrobust   LLR p-value:                 5.606e-84
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.6505      0.079     -8.267      0.000      -0.805      -0.496
pclass        -0.5174      0.118     -4.389      0.000      -0.748      -0.286
sibsp         -0.2268      0.095     -2.383      0.0

We got the same values. Nicely done. Now you know how the values are calculated under the hood in `statsmodels`, and how to determine them if you are using `sklearn`.